In [1]:
demand = [990,1980,3961,2971,1980]   
d=0  # d% shortage allowance
Y_b = [1.3086,1.3671,1.4183,1.4538,1.5122]           # Fabric yield (consumption rate) rate per garment of size 𝛽
f= 1.95 # Fabric cost 

U = 0.85
l_max= 20
e= .083   # Fabric end allowance 


if len(demand)!=len(Y_b):
    raise ValueError('number of sizes and number of fabric consumption does not match')

In [2]:
#Input variables (Marker)

M_d = 10                                # Average marker design time (minute)
z = 0.60                                # Printing speed per minute
vv = 0.42                                #Standard cost per minute in marker making floor (labor, machine & electricity)


In [3]:
#Input variables (Cutting Time)

T_G = 30                                  # General Preparation Time
xx= 9.908           # Average spreading speed in minutes after taking account for the idle strokes. 
T_M= 4           # Time for Placement of the marker
t_c= 4.73         # SMV of cutting time per garment pattern
T_S= 5           # preparation time for sticker placement
𝑡_𝑏 = 2.937                               # Standard minute value (SMV) of time takes to bundle.
𝑏 =  15                                   # pieces of garments in one bundle
ww = 0.30                                 # standard cost per minute in cutting floor (labor, machine & electricity)
P_min, P_max= 10,400

In [4]:
import numpy as np
import math
import pandas as pd
from copy import deepcopy
rng = np.random.default_rng()
import random
import time
import plotly.express as px
import plotly.graph_objects as go
from itertools import product

In [5]:
def Update_Res(R,GG,PP):
    for s in range(len(GG)): #Updating Residual within the while loop
        R=R-np.dot(GG[s],PP[s])
    return R

In [6]:
def Length(g_i_j):
    l_i = e+ np.dot(g_i_j,Y_b)/U
    return l_i


In [7]:
def Shortage_allowance(Q,d=0.01):
    temp=np.dot((1-d),Q)
    return [round(i) for i in temp]
Q_b= Shortage_allowance(demand,d)

# Q_b

In [8]:
from Heuristics import H1,H3,H5,H3_MB2016,H1_T2020

## Objective Function

In [9]:
def ObjectiveFunction (chromosome):
    
    G_a_b = chromosome['G']
    P_a = chromosome['P']
    Alpha = len(P_a)             # number of Sections
    

    '''                         Fabric Cost                      '''
        
    # Total fabric length = L # Total Fabric Cost = C_F

    l_a=[Length(G_a_b[alpha]) for alpha in range(Alpha) ] #Length function
    L= np.dot(l_a,P_a)       #Multiply then Sum
    C_F = L*f
    #print('Total Fabric Cost = C_F: ',C_F)
    
    
    '''                        Marker Cost                        '''
    
    #Marker Making Cost = C_M

    M_p_a = [(la-e)/z for la in l_a]     # devide each element of a 'l_a' by 'z'
    #M_p_a = Marker Printing time (minute) of section alpha 
 
    '''
    𝑟 = {1 ; 𝑖𝑓 𝑡h𝑒 𝑚𝑎𝑟𝑘𝑒𝑟 𝑖𝑠 𝑏𝑒𝑖𝑛𝑔 𝑢𝑠𝑒𝑑 𝑓𝑜𝑟 𝑡h𝑒 𝑓𝑖𝑟𝑠𝑡 𝑡𝑖𝑚𝑒 
        {0 ; 𝑖𝑓 𝑡h𝑒 𝑚𝑎𝑟𝑘𝑒𝑟 h𝑎𝑠 𝑏𝑒𝑒𝑛 𝑢𝑠𝑒𝑑 𝑏𝑒𝑓𝑜𝑟𝑒
    '''
    r=[]
    for i in range(Alpha):
        temp=0
        j=i-1
        while j>=0:
            if G_a_b[i]== G_a_b[j]:
                temp+=1
                break
            j-=1
        if temp==0:
            r.append(1)
        else:
            r.append(0)

    
    C_M = 0
    for α in range(Alpha):
        if l_a[α]>e:   # this makes sure that section has at least one garments 
            C_M += (M_d*r[α] + M_p_a[α])*vv       
    
    # 'if la>e' makes sure that the section contain at least one garments, 
    #  not all G_a_b values are zero
    
    
    '''                          Cutting Cost                            '''
        
    # Cutting Time of one section = T_T  # Total Cutting Cost = C_C
    
    #T_T  =T_G + T_F +T_M+ T_c+T_S +T_B
    
    T_C=[]   #Cutting time for every section
    for alpha in range(Alpha):
        T_C.append(sum(G_a_b[alpha])*t_c)
    
    T_F=[]  # Fab spreading time for each section
    for α in range(Alpha):
        T_F.append(l_a[α]*P_a[α]/xx)
    
    T_B=[] #Bundleing time for each section
    for α in range(Alpha):
        T_B.append(math.ceil(P_a[α]/b)*sum(G_a_b[α])*t_b)
    
    
    T_T_T = 0  #Total cutting time
    for α in range(Alpha):
        if l_a[α]>e:   # this makes sure that section has at least one garments
            T_T_T+=T_G+T_F[α]+T_M+T_C[α]+T_S+ T_B[α]
    
    
    C_C = T_T_T*ww  #Total cutting cost
    
    
    
    '''                              Total Cost                 '''
    # Total Cost = C_T = C_F + C_M + C_C
    
    return C_F+C_M+C_C

In [10]:
# ObjectiveFunction(Sol_1)

### Time spend

## Fitness Score

In [11]:
def Fitness(chromosome): 
  
    G_a_b= chromosome['G']
    P_a = chromosome['P']
    Beta= len(G_a_b[0])
    
    score= ObjectiveFunction(chromosome)
    #print('score:',score)
    fitness_score=score
    
                
    '''       Penalty for shortage production           '''
    R= Update_Res(R=demand,GG=G_a_b,PP=P_a)
    for beta in range(Beta):
        if R[beta]>0:
            s_penalty= R[beta]/sum(demand)
            fitness_score +=score*s_penalty/2 
    
    
    '''         Penalty for excess production           '''
    r=np.dot(1.02,demand)         # additional 2% allowance
    R= Update_Res(R=r,GG=G_a_b,PP=P_a)
    #print(R)
    for beta in range(Beta):
        if R[beta]<0:
            e_penalty= (-R[beta]/sum(demand))   # 2times than s_penalty
            fitness_score +=score*e_penalty   
            

    
    '''       double check if the solution is valid       '''
    res= Update_Res(R=Q_b,GG=G_a_b,PP=P_a)
    if max(res)>0:
        '''solution is unvalid'''
        print('demand constraint invalid',demand)
        fitness_score +=10000   #this will eventualy make the solution extinct.

    for i in G_a_b:
        if Length(i)>l_max:
            print('breaking !! fitness !! solution is not valid: ',i, Y_b, l_max,Length(i))
            fitness_score +=10000
    
    return fitness_score

# Fitness(Sol_1)    

## Function Initial Population Generation

In [12]:
def GeneratePopulation(pop_size):
    sol1=H3_MB2016(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,l_max=l_max,e=e)
    sol2=H1_T2020(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,l_max=l_max,e=e)
    P_of_S=[]
    for sol in (sol1,sol2):
        i=0
        for g in sol['G']:
            if Length(g)>l_max:
                i+=1
        if i==0:
            P_of_S.append(sol)
    
    while len(P_of_S)<pop_size:
        h=rng.integers(0,3)
        #print('h:',h)
        if h==0:
            sol= H1(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)

        elif h==1:
            sol=H3(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)

        else:
            sol=H5(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)
        i=0
        for g in sol['G']:
            if Length(g)>l_max:
                i+=1
        if i==0:
            P_of_S.append(sol)
    return P_of_S
Pool_of_Sol= GeneratePopulation(50)
len(Pool_of_Sol)

50

In [13]:
def S_with_F(p_o_s):
    for i in range(len(p_o_s)): 
        if 'F' not in p_o_s[i]:
            p_o_s[i]['F']=Fitness(p_o_s[i])
    return p_o_s

## PSO

### Cleaning section with zeros

In [14]:
def CleanZeros (Solution):
    j=0
    while j < len(Solution['G']):
        if max(Solution['G'][j])==0:
            Solution['G'].pop(j)
            Solution['P'].pop(j)
            continue
        j+=1

    #This is to make sure 
    if len(Solution['G'])!=len(Solution['P']):
        raise ValueError('P and G lengths are not same')
    
    return Solution

In [15]:
# CleanZeros(Sol_1)

## Velocity Update (Jarboui et al. 2008)

Lets assume 1st sol as X, 2nd Sol as Pbest, and 3rd Sol as Gbest

#### Now we have to calculate Y

##### Initial Velocity generator

In [16]:
def initial_velocity(Range, Sol): #Range is a list
    a,b= Range
    m=len(Sol['G'])
    
    #generate a random uniform array  [-a,b] of the same size of the solutions 
    
    v=(b-a) * np.random.random_sample(m) +a  #http://bit.ly/3To2OWe
    v=v.tolist()
    
    return {'V':v}

In [17]:
def Get_Y(X,GBest,PBest): #(Jarboui et al., 2008, p. 302)
    y=[]
    lens=[len(i) for i in [X['G'],GBest['G'],PBest['G']]]
    min_len=min(lens)
    
    for i in range(min_len):
        if X['G'][i]==GBest['G'][i] and X['G'][i]==PBest['G'][i]:
            y.append(random.choice([-1,1]))
        elif X['G'][i]==GBest['G'][i]:
            y.append(1)
        elif X['G'][i]==PBest['G'][i]:
            y.append(-1)
        else:
            y.append(0)
        
    return {'Y':y}

### Now we have to calculate Velocity

In [18]:
def New_V(Y,V,c1=1,c2=1,w=.75): #Parameter setting: (Jarboui et al., 2008, p. 306)

    for i in range(len(Y['Y'])):
        y=Y['Y'][i]
        v=V['V'][i]
        V['V'][i]= w*v+ rng.random()*c1*(-1-y)+rng.random()*c2*(1-y)
        
    return V

### Now we need to calculate λ

In [19]:
def Get_λ(Y,V):
    λ=[]
    for i in range(len(Y['Y'])):
        λ.append(Y['Y'][i]+V['V'][i])
    return {'λ':λ}
# λ=Get_λ(Y,V)
# λ

### Update X with Eq-10 (Jarboui et al., 2008, p. 303)

In [20]:
def Perturbation(xg,xp,R,p_rate):
    
    if rng.random()<p_rate:
        p1,p2=sorted([xp,min(P_max,max(P_min,max(R)))])
        xp= rng.integers(p1,p2+1)
    for j in range(len(xg)): #small purtubration (like mutaion)
        if rng.random()<p_rate:
            xg[j]=0
            if R[j]>0:
                temp= min(math.ceil(R[j]/xp),math.floor((l_max-Length(xg))/(Y_b[j]/U)))
                temp= max(0,temp)
                xg[j]=rng.integers(0,temp+1)
        if R[j]<=0 and xg[j]>0:
            xg[j]=0
            
    return xg,xp

def Update_X(X,GBest,PBest,λ, ϕ=0.5, p_rate=.1):

    XG=[]
    XP=[]
    R= Q_b
    for i in range(len(λ['λ'])):
        if λ['λ'][i] > ϕ:
            xg=GBest['G'][i]
            xp=GBest['P'][i]
                           
        elif λ['λ'][i] < -ϕ:
            xg=PBest['G'][i]
            xp=PBest['P'][i]

        else:
            xg,xp= Perturbation(xg=X['G'][i],xp=X['P'][i],R=R,p_rate=p_rate) #Perturbation function
        
        if max(xg)>0:    
            XG.append(xg)
            XP.append(xp)
            
        R= Update_Res(R=Q_b,GG=XG,PP=XP)
        if max(R)<=0:
            return {'G':XG,'P':XP}
  
    for i in range(len(λ['λ']), len(X['G'])):        
        xg,xp= Perturbation(xg=X['G'][i],xp=X['P'][i],R=R, p_rate=p_rate) #Perturbation function
        
        if max(xg)>0:
            XG.append(xg)
            XP.append(xp)
        R= Update_Res(R=Q_b,GG=XG,PP=XP)
        if max(R)<=0:
            return {'G':XG,'P':XP}

    h=H1(Q=R,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)
        
    g= h['G']
    p = h['P']    

    XG=XG+g
    XP=XP+p

    return {'G':XG,'P':XP}

In [21]:
def Update_dimension(XX,VV, in_vel_range=[-0.5,0.5]):
    mm= len(XX['G'])
    m= len(VV['V'])
    
    if mm <= m:
        return {'V':VV['V'][:m]}
    else:
        a,b= in_vel_range
        v=(b-a) * np.random.random_sample(mm-m) +a  #http://bit.ly/3To2OWe
        v=v.tolist()
        V=VV['V']+v
        return {'V':V}

In [22]:
def Get_Gbest(p_o_s):
    gbest=p_o_s[0]
    for i in range(len(p_o_s)):
        if p_o_s[i]['F']<gbest['F']:
            gbest= deepcopy(p_o_s[i])
    return gbest

# Main PSO

In [23]:
def PSO(swarmsize,iteration,ϕ=.7,c1=2,c2=2,w=1,p_rate=.2):
    
    P_of_S= GeneratePopulation(swarmsize)
    P_of_S= S_with_F(P_of_S)
    P_of_Pbest=P_of_S
    
    in_vel_range=[-ϕ,ϕ]
    P_of_Velocity= [initial_velocity(in_vel_range,P_of_S[i]) for i in range(len(P_of_S))]
    
    Gbest=Get_Gbest(P_of_S)
    sc=0
    bests=[Gbest['F']]
    for i in range(iteration):
        for j in range(len(P_of_S)):
            X=P_of_S[j]
            Pbest=P_of_Pbest[j]
            V= P_of_Velocity[j]
            Y= Get_Y(X=X,GBest=Gbest,PBest=Pbest)

            newV= New_V(Y=Y,V=V,c1=c1,c2=c2,w=w)
            
            λ= Get_λ(Y=Y,V=newV)

            newX= Update_X(X=X,GBest=Gbest,PBest=Pbest,λ=λ,ϕ=ϕ, p_rate=p_rate)
            newX['F']= Fitness(newX)

            P_of_S[j]=deepcopy(newX)
            
            newV= Update_dimension(XX=newX,VV= newV, in_vel_range=in_vel_range)
            P_of_Velocity[j]= deepcopy(newV)
            
            if newX['F'] < Pbest['F']:
                P_of_Pbest[j]= deepcopy(newX)
            if newX['F'] < Gbest['F']:
                Gbest=deepcopy(newX)
                sc=i
        #print(Gbest, Fitness(Gbest))
        bests.append(Gbest['F'])
    
    # xx=[i for i in range(len(bests))]
    # fig=px.line(x=xx,
    #             y=bests,
    #             title=f'swarmsize={swarmsize},iteration= {iteration},ϕ={ϕ},c1= {c1},c2={c2},w={w}, Gbest={bests[-1]}',
    #             labels=dict(x='iteration',y='fitness'))
    # fig.show()
    
    return CleanZeros(Gbest),sc
# PSO(swarmsize=150,iteration=250)

## Parameter Tuning

In [24]:
ϕ=[.9,.7,.5,.3]
c1=[2,1.5,1]
c2=[2,1.5,1]
w=[.9,.6,.4]
p_rate=[.3,.2,.1]
iteration=product(ϕ,c1,c2,w,p_rate)
df = pd.DataFrame(columns=['demand','sol','ϕ','c1','c2','w','p_rate','Fitness','Runtime','sc'])

for ϕ,c1,c2,w,p_rate in list(iteration):
    Dataset={ 'demands':[[10,21,33,41,39,40,30],
                         [193,501,1018,1249,998,564,250,128],
                         [990,1980,3961,2971,1980]],
             'consumption':[[1.5073,1.5685,1.6126,1.6704,1.7182,1.7911,1.8308],
                            [1.3350,1.3998,1.4356,1.4826,1.5440,1.5878,1.6313,1.6908],
                            [1.3086,1.3671,1.4183,1.4538,1.5122]],
             'price':[3.10,2.90,1.95]
             }
    for i in range(len(Dataset['demands'])):
        demand=Dataset['demands'][i]
        Q_b= Shortage_allowance(demand,d)
        Y_b=Dataset['consumption'][i]
        f=Dataset['price'][i]
        for j in range(5):
            time_start = time.perf_counter()
            Gbest,sc =PSO(swarmsize=200,iteration=180,c1=c1,c2= c2,ϕ =ϕ ,w=w,p_rate=p_rate)
            time_elapsed = (time.perf_counter() - time_start)
            
            df=df.append({'demand':demand,'sol':Gbest,'ϕ':ϕ,'c1':c1,'c2': c2,'w': w,'p_rate': p_rate,
                          'Fitness':Fitness(Gbest), 'Runtime': time_elapsed,'sc':sc}, 
                         ignore_index=True)
#     df.to_csv('PSO_GridSearch_All_7_jan_23.csv')

KeyboardInterrupt: 

In [26]:
df.head()

,demand,sol,ϕ,c1,c2,w,p_rate,Fitness,Runtime,sc
0,"[10, 21, 33, 41, 39, 40, 30]","{'G': [[0, 0, 0, 1, 1, 1, 1], [1, 0, 0, 0, 1, ...",0.9,2,2,0.9,0.3,1467.680616,10.094606,126
1,"[10, 21, 33, 41, 39, 40, 30]","{'G': [[0, 0, 0, 1, 0, 2, 0], [0, 1, 3, 1, 1, ...",0.9,2,2,0.9,0.3,1488.284598,10.989229,16
2,"[10, 21, 33, 41, 39, 40, 30]","{'G': [[1, 1, 0, 0, 0, 4, 1], [0, 1, 3, 1, 0, ...",0.9,2,2,0.9,0.3,1479.184048,8.019372,22
3,"[10, 21, 33, 41, 39, 40, 30]","{'G': [[1, 1, 0, 0, 4, 0, 3], [0, 0, 0, 1, 0, ...",0.9,2,2,0.9,0.3,1476.378958,7.870951,5
4,"[10, 21, 33, 41, 39, 40, 30]","{'G': [[1, 0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, ...",0.9,2,2,0.9,0.3,1494.045095,13.734340,69


In [27]:
def PSO_with_sc(swarmsize,iteration,ϕ=.7,c1=2,c2=2,w=1,p_rate=.2,sc=100):
    
    P_of_S= GeneratePopulation(swarmsize)
    P_of_S= S_with_F(P_of_S)
    P_of_Pbest=P_of_S
    
    in_vel_range=[-ϕ,ϕ]
    P_of_Velocity= [initial_velocity(in_vel_range,P_of_S[i]) for i in range(len(P_of_S))]
    
    Gbest=Get_Gbest(P_of_S)
    it=0
    bests=[Gbest['F']]
    for i in range(iteration):
        for j in range(len(P_of_S)):
            X=P_of_S[j]
            Pbest=P_of_Pbest[j]
            V= P_of_Velocity[j]
            Y= Get_Y(X=X,GBest=Gbest,PBest=Pbest)

            newV= New_V(Y=Y,V=V,c1=c1,c2=c2,w=w)
            
            λ= Get_λ(Y=Y,V=newV)

            newX= Update_X(X=X,GBest=Gbest,PBest=Pbest,λ=λ,ϕ=ϕ, p_rate=p_rate)
            newX['F']= Fitness(newX)

            P_of_S[j]=deepcopy(newX)
            
            newV= Update_dimension(XX=newX,VV= newV, in_vel_range=in_vel_range)
            P_of_Velocity[j]= deepcopy(newV)
            
            if newX['F'] < Pbest['F']:
                P_of_Pbest[j]= deepcopy(newX)
            if newX['F'] < Gbest['F']:
                Gbest=deepcopy(newX)

        #print(Gbest, Fitness(Gbest))
        bests.append(Gbest['F'])
        if bests[-1] < bests[-2]:
            it=0
        else:
            it+=1
        if it>sc:
            break
    
#     xx=[i for i in range(len(bests))]
#     fig=px.line(x=xx,
#                 y=bests,
#                 title=f'swarmsize={swarmsize},iteration= {iteration},ϕ={ϕ},c1= {c1},c2={c2},w={w}, Gbest={bests[-1]}',
#                 labels=dict(x='iteration',y='fitness'))
#     fig.show()
    
    return CleanZeros(Gbest)
# PSO(swarmsize=150,iteration=250)

## Parameter tuning for population and iterartion

In [28]:
swarmsize=[160,180,200,250]
iteration=[160,180,200,250]
df = pd.DataFrame(columns=['demand','sol','swarmsize','iteration','Fitness','Runtime'])
for swarmsize,iteration in product(swarmsize,iteration):
    Dataset={ 'demands':[[10,21,33,41,39,40,30],
                         [193,501,1018,1249,998,564,250,128],
                         [990,1980,3961,2971,1980]],
             'consumption':[[1.5073,1.5685,1.6126,1.6704,1.7182,1.7911,1.8308],
                            [1.3350,1.3998,1.4356,1.4826,1.5440,1.5878,1.6313,1.6908],
                            [1.3086,1.3671,1.4183,1.4538,1.5122]],
             'price':[3.10,2.90,1.95]
             }
    
    
    c1,c2,ϕ,w,p_rate,sc= (2,2,0.5,0.4,0.2,90) #parameters
    for i in range(len(Dataset['demands'])):
        demand=Dataset['demands'][i]
        Q_b= Shortage_allowance(demand,d)
        Y_b=Dataset['consumption'][i]
        f=Dataset['price'][i]
        for j in range(5):
            time_start = time.perf_counter()
            Gbest= PSO_with_sc(swarmsize=swarmsize,iteration=iteration,ϕ=ϕ,c1=c1,c2=c2,w=w,p_rate=p_rate,sc=sc)
            time_elapsed = (time.perf_counter() - time_start)
            
            df=df.append({'demand':demand,'sol':Gbest,'swarmsize':swarmsize ,'iteration':iteration,
                          'Fitness':Fitness(Gbest), 'Runtime': time_elapsed}, 
                         ignore_index=True)
    #df.to_csv('PSO_pop_iter_tuning_7_Jan_23.csv')

KeyboardInterrupt: 

In [29]:
df.head()

,demand,sol,swarmsize,iteration,Fitness,Runtime
0,"[10, 21, 33, 41, 39, 40, 30]","{'G': [[1, 1, 0, 0, 0, 4, 3], [0, 1, 3, 1, 0, ...",160,160,1479.184048,3.542095
1,"[10, 21, 33, 41, 39, 40, 30]","{'G': [[1, 1, 0, 0, 0, 4, 3], [0, 1, 3, 0, 0, ...",160,160,1481.798456,3.672086


## Selected Parameters : 
* c1=c2= 2, 
* ϕ= 0.5, 
* w= 0.4, 
* p_rate=0.2 
* sc=90 (stoping criteria)
* swarmsize= 250 
* iteration= 200

## Applying PSO to solve Industry problem

In [30]:
path='/Users/sharif-al-mahmud/Library/Mobile Documents/com~apple~CloudDocs/Thesis/Data/DataSets.xlsx'

df=pd.read_excel(path,sheet_name='Sheet3',index_col=[0])

df=df.T

df.columns = df.columns.str.strip()

col=list(df.columns)
col= sorted(set(col), key=col.index)

df.head()

Style short,ST1,ST1,ST1,ST1,ST1,ST1,ST1,ST1,ST2,ST2,...,ST34,ST34,ST35,ST35,ST35,ST35,ST35,ST35,ST35,ST35
Style,Girl's High Rise Wide Leg Belted Trouser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Falcon lake shorty short,NaN,...,NaN,NaN,Mens Pull on Jogger,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Demand,136,354,743,928,738,423,188,91,872,1743,...,48,34,0,10,21,33,41,39,40,30
Consumption,2.0581,2.128,2.2061,2.2688,2.3334,2.4134,2.4793,2.597,0.612,0.6315,...,1.6377,1.7032,1.4688,1.5073,1.5685,1.6126,1.6704,1.7182,1.7911,1.8308
Price/unit length,3.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.51,NaN,...,NaN,NaN,3.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
print(col)

['ST1', 'ST2', 'ST3', 'ST4', 'ST5', 'ST6', 'ST7', 'ST8', 'ST9', 'ST10', 'ST11', 'ST12', 'ST13', 'ST14', 'ST15', 'ST16', 'ST17', 'ST18', 'ST19', 'ST20', 'ST21', 'ST22', 'ST23', 'ST24', 'ST25', 'ST26', 'ST27', 'ST28', 'ST29', 'ST30', 'ST31', 'ST32', 'ST33', 'ST34', 'ST35']


In [35]:
df_result_PSO = pd.DataFrame(columns=['style','demand','sol','Fitness','Cost','Runtime'])

c1,c2= 2,2 
ϕ= 0.5
w= 0.4 
p_rate=0.2 
sc=90 #stoping criteria
swarmsize= 250 
iteration= 200  #parameters
for j in range(5):
    for i in col:
        demand=df[i].loc['Demand'].to_list()
        demand=list(map(int,demand))
        #print('demand=',demand)

        d=0
        Q_b= Shortage_allowance(demand,d)

        consumption=df[i].loc['Consumption'].to_list()
        Y_b=list(map(float,consumption))
        #print('consumption=',Y_b)

        price=df[i].loc['Price/unit length'].to_list()
        price=list(map(float,price))
        f=price[0]

        print(i,demand,Q_b,Y_b,f)
        time_start = time.perf_counter()
        Gbest = PSO_with_sc(swarmsize=swarmsize,iteration=iteration,ϕ=ϕ,c1=c1,c2=c2,w=w,p_rate=p_rate,sc=sc)
        print(Gbest['F'])
        time_elapsed = (time.perf_counter() - time_start)
        df_result_PSO=df_result_PSO.append({'style':i,'demand':demand,'Cost': ObjectiveFunction(Gbest),'sol':Gbest,
                                            'Fitness':Gbest['F'],'Runtime': time_elapsed},
                                           ignore_index=True)
    #df_result_PSO.to_excel('/Users/sharif-al-mahmud/Library/Mobile Documents/com~apple~CloudDocs/Thesis/Data/PSO_industry_data_7_jan_23.xlsx')

ST1 [136, 354, 743, 928, 738, 423, 188, 91] [136, 354, 743, 928, 738, 423, 188, 91] [2.0581, 2.128, 2.2061, 2.2688, 2.3334, 2.4134, 2.4793, 2.597] 3.56
35433.93045123417
ST2 [872, 1743, 3486, 2614, 1743] [872, 1743, 3486, 2614, 1743] [0.612, 0.6315, 0.6499, 0.6721, 0.6922] 1.51
13249.578816016292
ST3 [12, 67, 131, 187, 191, 138, 79, 27] [12, 67, 131, 187, 191, 138, 79, 27] [0.7198, 0.7352, 0.7614, 0.7878, 0.8146, 0.8423, 0.8579, 0.8985] 2.4269860115743813
2254.7636385334877
ST4 [72, 187, 392, 489, 389, 223, 99, 48] [72, 187, 392, 489, 389, 223, 99, 48] [1.2538, 1.2857, 1.3259, 1.3751, 1.4189, 1.4601, 1.502, 1.5709] 2.2
7321.357281702249
ST5 [57, 147, 310, 386, 307, 176, 78, 38] [57, 147, 310, 386, 307, 176, 78, 38] [1.3972, 1.4533, 1.4987, 1.548, 1.603, 1.6415, 1.7132, 1.7644] 3.5
10116.516310316083
ST6 [242, 628, 1286, 1585, 1264, 717, 318, 160] [242, 628, 1286, 1585, 1264, 717, 318, 160] [1.2863, 1.3183, 1.3587, 1.409, 1.4593, 1.5107, 1.5423, 1.6055] 1.95
21243.572164550333
ST7 [990,

KeyboardInterrupt: 

In [36]:
df_result_PSO.head()

,style,demand,sol,Fitness,Cost,Runtime
0,ST1,"[136, 354, 743, 928, 738, 423, 188, 91]","{'G': [[0, 0, 0, 6, 0, 0, 0, 1], [1, 0, 5, 0, ...",35433.930451,35433.930451,33.469889
1,ST2,"[872, 1743, 3486, 2614, 1743]","{'G': [[2, 4, 8, 6, 4], [1, 1, 3, 1, 1], [0, 1...",13249.578816,13249.578816,7.541488
2,ST3,"[12, 67, 131, 187, 191, 138, 79, 27]","{'G': [[1, 0, 0, 0, 8, 0, 1, 0], [0, 0, 2, 1, ...",2254.763639,2254.059245,25.102082
3,ST4,"[72, 187, 392, 489, 389, 223, 99, 48]","{'G': [[0, 0, 8, 0, 0, 3, 0, 1], [0, 1, 0, 7, ...",7321.357282,7321.357282,27.059645
4,ST5,"[57, 147, 310, 386, 307, 176, 78, 38]","{'G': [[1, 0, 8, 0, 0, 0, 1, 1], [1, 0, 1, 3, ...",10116.516310,10116.516310,23.788619


## Cost of current solutions

In [37]:
ind_sol=[{ 'Name': 'W-Trousers(Crs) MISSY-REGULAR',
          'G' : [[0,0,1,1,2,2,1,1], [0,1,1,1,1,1,1,0], [2,1,2,1,1,1,0,0], [0,2,0,3,1,0,0,0], [0,0,0,1,1,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,1,0,0,0,0]],
          'P' : [400,200,400,350,100,50,50],
          'demand': [800, 1300, 1400, 2200, 1900, 1400, 600, 400],
          'Y_b': [0.9596, 1.0025, 1.0377, 1.0653, 1.1003, 1.1281, 1.1852, 1.2142],
          'price': 1.24
},
         
{'Name': 'Women Shorts 036_BLACK',
 'G' : [[0,0,1,2,2,2,1,0], [0,2,2,0,0,0,1,1], [0,2,0,1,0,0,0,0], [0,0,0,1,1,0,0,0], [0,0,0,0,0,0,1,1], [1,0,0,0,0,0,1,0]],
 'P' : [94,60,26,40,19,18],
 'demand': [18, 172, 214, 254, 227, 187, 187, 79],
  'Y_b': [0.9589, 0.9855, 1.0252, 1.0535, 1.0888, 1.1235, 1.152, 1.1998],
  'price': 1.5
},

{
'Name': 'Women Shorts 616_BLUE_HAZE',
'G' : [[0,0,1,2,2,2,1,0], [0,2,2,0,0,0,1,1], [0,2,0,1,0,0,1,0], [0,0,0,1,1,0,0,0],[0,1,1,0,0,0,0,0], [1,0,0,0,0,0,0,0]], 
'P' : [82,46,21,17,10,13],
'demand': [13, 141, 175, 201, 181, 164, 149, 46],
'Y_b': [0.9589, 0.9855, 1.0252, 1.0535, 1.0888, 1.1235, 1.152, 1.1998],
'price': 1.38
},


{
'Name': 'Women Shorts 210_PALE_ROSETTE',
'G' : [[0,0,1,2,2,2,1,0], [0,2,2,0,0,0,1,1], [0,0,0,1,1,0,0,0],[0,2,0,1,0,0,1,0],[0,0,1,2,0,0,0,0], [1,1,0,0,1,0,0,0]],
'P' : [65,36,15,10,10,15],
'demand': [14, 101, 139, 154, 145, 129, 108, 36],
'Y_b': [0.9589, 0.9855, 1.0252, 1.0535, 1.0888, 1.1235, 1.152, 1.1998],
'price': 1.46
},


{'Name': 'Falcon lake Shorty short',
'G' : [[0,2,2,2,2],[0,2,2,2,2],[2,0,2,2,0], [0,1,1,1,1],[0,0,4,0,0],[0,0,2,0,0], [1,0,0,1,0], [0,0,1,0,0]],
'P' : [400,400,400,143,180,100,72,23],
'demand': [872, 1743, 3486, 2614, 1743],
'Y_b': [0.612, 0.6315, 0.6499, 0.6721, 0.6922],
'price': 1.51
},


{'Name': 'Pull-on Short  Backpacker Trail',
'G' : [[0,0,0,1,1,1,1,0], [0,0,1,1,1,1,0,0], [0,1,1,1,1,0,0,0], [0,0,1,0,0,0,0,1], [1,1,0,0,0,0,0,0], [0,1,0,0,1,0,0,1]],
'P' : [79,59,49,23,12,10],
 'demand': [12, 67, 131, 187, 191, 138, 79, 27],
  'Y_b': [0.7198, 0.7352, 0.7614, 0.7878, 0.8146, 0.8423, 0.8579, 0.8985],
  'price': 2.4269860115743813
},

{'Name': 'Camper Shorty Woman',
'G' : [[3,2,2,1], [4,1,1,0], [4,1,1,1],[2,1,1,0], [0,0,0,1]],
'P' : [400,100,98,52,26],
 'demand': [2096, 1048, 1048, 524],
  'Y_b': [0.7434, 0.7662, 0.8028, 0.8239],
  'price': 3.3
},

{'Name': 'Polyester Pants Mens' ,
'G' :[[0,2,2,2,2], [2,0,2,2,0], [0,2,2,2,2],[0,0,1,1,0], [0,1,2,0,1], [1,0,2,1,0], [0,0,1,0,0],[1,0,0,0,0]],
'P' : [400,400,400,400,380,171,59,19],
 'demand': [990, 1980, 3961, 2971, 1980],
  'Y_b': [1.3086, 1.3671, 1.4183, 1.4538, 1.5122],
  'price': 1.95
},

{'Name': "Everyday Trouser Ladies",
'G' : [[3,2,2,1], [4,1,1,0], [4,1,1,1],[2,1,1,0], [0,0,0,1]],
'P' : [400,100,98,52,26],
 'demand': [2096, 1048, 1048, 524],
  'Y_b': [1.38, 1.4251, 1.4665, 1.5237],
  'price': 2.54
},


{'Name': "Pant (9117) Men's" ,
 'G' : [[0,0,0,1,1,1,1,0], [0,0,1,1,1,1,0,0], [0,0,1,0,0,0,0,1], [0,1,1,0,0,0,0,0], [1,1,0,0,0,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,1,0,0,0], [0,1,0,0,0,0,0,0]],
 'P' : [119,22,60,37,18,10,11,10],
 'demand': [18, 58, 119, 141, 152, 145, 119, 60],
  'Y_b': [1.2906, 1.3286, 1.3911, 1.4239, 1.4816, 1.5073, 1.589, 1.625],
  'price': 4.132843114970044
},

{'Name': 'Pant (9151) Men',
'G' : [[0,0,0,1,1,1,1,0], [0,0,1,1,1,1,0,0], [0,1,1,1,1,0,0,0], [0,0,1,0,0,0,0,1], [0,1,1,0,0,0,0,0], [0,0,0,0,1,0,0,0], [0,1,0,0,0,0,0,0], [1,0,0,0,0,0,0,0]],
'P' : [67,24,11,23,18,10,22,10],
 'demand': [10, 51, 76, 102, 104, 91, 67, 23],
  'Y_b': [1.2303, 1.2673, 1.328, 1.3591, 1.4089, 1.4538, 1.5202, 1.5584],
  'price': 4.132843114970044
},

{'Name': 'Relaxed Chino Trouser Ladies',
'G' : [[0,1,1,1,1,1,1,0], [0,0,1,2,2,1,0,0], [1,0,1,0,1,0,0,1], [0,1,0,1,0,0,0,0], [0,0,2,0,0,0,0,0],[1,0,0,1,0,0,0,0],[0,0,0,0,0,0,0,1], [0,0,0,1,0,0,0,0]],
'P' : [250,314,120,251,167,73,10,47],
  'demand': [193, 501, 1018, 1249, 998, 564, 250, 128],
  'Y_b': [1.335, 1.3998, 1.4356, 1.4826, 1.544, 1.5878, 1.6313, 1.6908],
  'price': 2.9
},

{'Name': "Ultra-Comfort Jegging (SS'23) Ladies",
'G' : [[0,1,1,1,1,1,1,0], [0,0,1,2,2,1,0,0], [1,0,1,0,1,0,0,1], [0,1,0,1,0,0,0,0], [0,0,2,0,0,0,0,0], [1,0,0,1,0,0,0,0],[0,0,0,0,0,0,0,1], [0,0,0,1,0,0,0,0]],
'P' : [162,202,69,143,104,48,10,42],
 'demand': [117, 305, 640, 799, 635, 364, 162, 78],
  'Y_b': [1.1327, 1.1655, 1.2074, 1.25, 1.2918, 1.3327, 1.3676, 1.4328],
  'price': 3.100000012056481
},
         
{ 'Name': 'Mens Pull on Jogger',
'G' : [[0,0,0,0,1,1,0,0], [0,0,0,1,0,0,1,0], [0,0,1,0,0,0,0,1], [0,1,0,0,0,0,0,1],[0,0,0,0,0,0,1,0]],
'P' : [41,33,21,10,10],
'demand': [0, 10, 21, 33, 41, 39, 40, 30],
  'Y_b': [1.4688, 1.5073, 1.5685, 1.6126, 1.6704, 1.7182, 1.7911, 1.8308],
  'price': 3.1
},
{'Name': "Men's Pull on Trousers (SS'23)",
'G' : [[0,1,1,1,1,1,1,0], [0,0,1,2,2,1,0,0], [1,0,1,0,1,0,0,1], [0,1,0,1,0,0,0,0], [0,0,2,0,0,0,0,0], [1,0,0,1,0,0,0,0], [0,0,0,0,0,0,0,1],[0,0,0,1,0,0,0,0]],
'P' : [136,169,59,120,87,39,10,37],
 'demand': [98, 256, 537, 670, 533, 305, 136, 66],
  'Y_b': [1.4684, 1.5185, 1.5701, 1.6195, 1.6612, 1.7176, 1.7892, 1.8412],
  'price': 2.75
},
{
'G' : [[0,1,1,1,1,1,1,0], [0,0,1,2,2,1,0,0], [1,0,1,0,1,0,0,0], [0,1,0,1,0,0,0,0], [0,0,2,0,0,0,0,0], [1,0,0,1,0,0,0,0], [0,0,0,0,0,0,0,1], [0,0,0,1,0,0,0,1]],
'P' : [42,52,18,37,27,12,10,11],
 'Name': "Mens Stretch 5 Pkt Trousers (AW'22)",
  'demand': [30, 79, 165, 206, 164, 94, 42, 20],
  'Y_b': [1.3753, 1.403, 1.4646, 1.5129, 1.551, 1.6009, 1.6605, 1.7158],
  'price': 3.7
},
         
{'Name': 'Men Core Chino Trousers (Slim/Straight- SS23)',
'G' : [[0,1,1,1,1,1,1,0], [0,0,1,2,2,1,0,0], [1,0,1,0,1,0,0,1], [0,1,0,1,0,0,0,0], [0,0,2,0,0,0,0,0], [1,0,0,1,0,0,0,0], [0,0,0,0,0,0,0,1], [0,0,0,1,0,0,0,0]],
'P' : [199,251,94,198,133,58,10,39],
 'demand': [152, 397, 810, 996, 795, 450, 199, 101],
  'Y_b': [1.3532, 1.4033, 1.452, 1.4904, 1.5433, 1.5868, 1.6336, 1.6976],
  'price': 2.75
},
         
{'Name': 'Stretch SlimFit Chino Men',
'G' : [[0,1,1,1,1,1,1,0], [0,0,1,2,2,1,0,0], [1,0,1,0,1,0,0,1], [0,1,0,1,0,0,0,0], [0,0,2,0,0,0,0,0], [1,0,0,1,0,0,0,0], [0,0,0,0,0,0,0,1], [0,0,0,1,0,0,0,0]],
'P' :[209,261,89,184,134,62,12,54],
 'demand': [151, 393, 826, 1031, 820, 470, 209, 101],
  'Y_b': [1.3471, 1.3884, 1.441, 1.4942, 1.5484, 1.5901, 1.6413, 1.7019],
  'price': 2.85
},

{'Name': 'Men Golf Essential Pant',
'G' : [[0,1,1,1,1,1,1,0], [0,0,1,2,2,1,0,0], [1,0,1,0,1,0,0,1], [0,1,0,1,0,0,0,0], [0,0,2,0,0,0,0,0], [1,0,0,1,0,0,0,0], [0,0,0,0,0,0,0,1], [0,0,0,1,0,0,0,0]],
'P' :[99,124,42,88,64,30,10,24],
 'demand': [72, 187, 392, 489, 389, 223, 99, 48],
  'Y_b': [1.2538, 1.2857, 1.3259, 1.3751, 1.4189, 1.4601, 1.502, 1.5709],
  'price': 2.2
},


{'Name': 'Heritage 5 Pkt Pant',
'G' : [[0,1,1,1,1,1,1,0], [0,0,1,2,2,1,0,0], [1,0,1,0,1,0,0,1], [0,1,0,1,0,0,0,0], [0,0,2,0,0,0,0,0], [1,0,0,1,0,0,0,0], [0,0,0,0,0,0,0,1], [0,0,0,1,0,0,0,0]],
'P' :[78,98,33,69,51,24,10,19],
 'demand': [57, 147, 310, 386, 307, 176, 78, 38],
  'Y_b': [1.3972, 1.4533, 1.4987, 1.548, 1.603, 1.6415, 1.7132, 1.7644],
  'price': 3.5
},
         

{'Name': 'Mens Pants Expandable Waist(Sustainable)',
'G' : [[0,1,1,1,1,1,1,0], [0,0,1,2,2,1,0,0], [1,0,1,0,1,0,0,1], [0,1,0,1,0,0,0,0], [0,0,2,0,0,0,0,0], [1,0,0,1,0,0,0,0], [0,0,0,0,0,0,0,1], [0,0,0,1,0,0,0,0]],
'P' :[318,399,148,310,211,94,12,65],
  'demand': [242, 628, 1286, 1585, 1264, 717, 318, 160],
  'Y_b': [1.2863, 1.3183, 1.3587, 1.409, 1.4593, 1.5107, 1.5423, 1.6055],
  'price': 1.95
},


{'Name': 'Girls High Rise Wide Leg Belted Trouser',
'G' : [[0,1,1,1,1,1,1,0], [0,0,1,2,2,1,0,0], [1,0,1,0,1,0,0,1], [0,1,0,1,0,0,0,0], [0,0,2,0,0,0,0,0], [1,0,0,2,0,0,0,0], [0,0,0,0,0,0,0,1]],
'P' :[188,235,80,166,120,56,11],
 'demand': [136, 354, 743, 928, 738, 423, 188, 91],
  'Y_b': [2.0581, 2.128, 2.2061, 2.2688, 2.3334, 2.4134, 2.4793, 2.597],
  'price': 3.56
},


{
'Name': 'Ladies EPP Jeans',
'G' : [[0,1,1,1,1,1,1,0], [0,0,1,2,2,1,0,0], [1,0,1,0,1,0,0,1], [0,1,0,1,0,0,0,0], [0,0,2,0,0,0,0,0], [1,0,0,2,1,0,0,0], [0,0,1,1,1,4,0,1], [0,0,0,1,0,0,0,0]],
'P' :[366,400,170,357,261,107,14,64],
    'demand': [277, 717, 1472, 1815, 1449, 822, 365, 184],
  'Y_b': [1.3013, 1.3536, 1.3944, 1.4424, 1.4954, 1.5309, 1.5993, 1.6499],
  'price': 2.47000000251445
},

{'Name': 'Boys 5Pkt Denim Pants(repl)',
'G' : [[0,1,1,1,1,1,1,0], [0,0,1,2,2,1,0,0], [1,0,1,0,1,0,0,1], [0,1,0,1,0,0,0,0], [0,0,2,0,0,0,0,0], [1,0,0,2,0,0,0,0], [0,0,0,0,0,0,0,1]],
'P' :[73,91,32,65,47,21,10],
 'demand': [53, 138, 289, 361, 287, 164, 73, 35],
  'Y_b': [0.8641, 0.8995, 0.937, 0.9633, 0.9966, 1.0213, 1.0669, 1.0948],
  'price': 2.737822124460086
},
        
{ 'Name': 'Mens Table Chino Short',

'G' : [[1,2,1,1,1],[1,1,1,1,0],[1,0,1,0,0], [0,0,0,1,0], [1,0,0,0,0]],
'P' :[120,51,84,14,17],
 'demand': [272, 291, 255, 185, 120],
  'Y_b': [0.9493, 0.9781, 1.0079, 1.049, 1.0825],
  'price': 2.65
},

{
'Name': 'Flex Pull On Shorts',
'G' : [[1,2,1,1,1],[1,1,1,1,0],[1,0,1,0,0], [0,0,0,1,0], [1,0,0,0,0]],
'P' :[104,43,73,12,14],
'demand': [234, 251, 220, 159, 104],
'Y_b': [0.8402, 0.8573, 0.8911, 0.9214, 0.9557],
'price': 3.85
},

{
'Name': 'AE Chino Pant',
'G' : [[1,2,1,1,1],[1,1,1,1,0],[1,0,1,0,0], [0,0,0,1,0], [1,0,0,0,0]],
'P' :[ 120,51,84,14,17],
'demand': [272, 291, 255, 185, 120],
  'Y_b': [1.2572, 1.3055, 1.3482, 1.4003, 1.4397],
  'price': 3.43
},

{
'Name': "GT Mens Lightweight Oxford Shorts",
'G' : [[0,1,2,2,1], [2,2,0,0,2], [1,1,0,0,0],[0,0,2,2,0], [1,0,1,0,0], [1,0,0,0,0]], 
'P' :[400,144,108,140,36,72],
'demand': [504, 792, 1116, 1080, 684],
  'Y_b': [0.9464, 0.9795, 1.0118, 1.0434, 1.0775],
  'price': 3.42
},

{
'Name': "Girl's Pull On Shorts (S1'23)",
  'demand': [252, 288, 288, 288, 216],
  'Y_b': [0.5125, 0.5334, 0.5516, 0.5669, 0.5846],
  'price': 2.492677899887066,

'G' : [[0,1,2,2,1], [2,1,0,0,1], [1,1,0,0,0], [1,0,0,0,0]], 
'P' :[144,72,72,36],
},
         
{'Name': "Men's Benton Shorts (S1'23)",
  'demand': [684, 1080, 1476, 1476, 936],
  'Y_b': [0.9586, 0.9888, 1.0258, 1.0518, 1.0819],
  'price': 2.51,
'G' : [[0,1,2,2,1], [2,1,0,0,1], [1,1,0,0,0],[0,1,2,2,1],[2,0,0,0,0],[1,0,0,0,0]],
'P' :[400,198,144,338,50,44]
    
},
         
{'Name': "Boy's Straight Fit Denim",
  'demand': [360, 396, 432, 396, 288],
  'Y_b': [0.9552, 0.9781, 1.0168, 1.0473, 1.0764],
  'price': 2.6299999982047053,
'G' : [[0,1,2,2,1], [2,1,0,0,1], [1,1,0,0,0], [1,0,1,0,0], [1,0,0,0,0]], 
'P' :[198,90,108,36,36]
},

{'Name': 'Woman Jean (8076)',
  'demand': [33, 167, 269, 269, 135, 0],
  'Y_b': [1.2294, 1.253, 1.3182, 1.3472, 1.3855, 1.4375],
  'price': 3.6161865464796987,
'G' : [[0,1,2,2,1,0],[1,1,0,0,0,0]],
'P' :[135,33]
} ,    
{
    'Name': 'Men Jeans',
    'G':[[0,0,0,0,1,1,0,0],[0,0,0,1,0,0,1,0],[0,0,1,0,0,0,0,1],[0,1,0,0,0,0,0,1],[0,0,0,0,1,0,1,0],[1,0,0,0,0,0,0,0]],
    'P':[77,65,34,14,10,10],
     'demand': [2, 14, 34, 65, 86, 77, 71, 45],
  'Y_b': [1.4082, 1.4678, 1.5187, 1.5654, 1.6174, 1.6717, 1.7189, 1.782],
  'price': 3.55
}
        
        ]

In [39]:
df_ind_sol=pd.DataFrame(columns=['style','demand','fitness','cost'])
for i in ind_sol:
    demand=i['demand']
    Q_b= Shortage_allowance(demand,0.01)
    Y_b=i['Y_b']
    f=i['price']
    
    fitness=Fitness(i)
    cost=ObjectiveFunction(i)
    style=i["Name"]
    df_ind_sol=df_ind_sol.append({'style':style,'demand':demand, 'fitness':fitness,'cost':cost},ignore_index=True)

In [42]:
df_ind_sol

,style,demand,fitness,cost
0,W-Trousers(Crs) MISSY-REGULAR,"[800, 1300, 1400, 2200, 1900, 1400, 600, 400]",16990.866320,16990.866320
1,Women Shorts 036_BLACK,"[18, 172, 214, 254, 227, 187, 187, 79]",2876.624925,2876.066048
2,Women Shorts 616_BLUE_HAZE,"[13, 141, 175, 201, 181, 164, 149, 46]",2189.678469,2178.116133
3,Women Shorts 210_PALE_ROSETTE,"[14, 101, 139, 154, 145, 129, 108, 36]",1987.864522,1894.340081
4,Falcon lake Shorty short,"[872, 1743, 3486, 2614, 1743]",13473.142535,13473.142535
5,Pull-on Short Backpacker Trail,"[12, 67, 131, 187, 191, 138, 79, 27]",2238.517224,2211.143691
6,Camper Shorty Woman,"[2096, 1048, 1048, 524]",14857.068014,14857.068014
7,Polyester Pants Mens,"[990, 1980, 3961, 2971, 1980]",40738.541330,40738.541330
8,Everyday Trouser Ladies,"[2096, 1048, 1048, 524]",20919.135042,20919.135042
9,Pant (9117) Men's,"[18, 58, 119, 141, 152, 145, 119, 60]",6345.773770,6276.668577


In [ ]:
#df_ind_sol.to_excel('/Users/sharif-al-mahmud/Library/Mobile Documents/com~apple~CloudDocs/Thesis/Data/ind_sol2.xlsx')